### Clean up and Optimise Spatial Feature Detection


In [1]:
from __future__ import division, print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
from tempfile import mkdtemp, mkstemp
from subprocess import call, check_output, Popen, PIPE
from os.path import join, isfile, isdir, exists

In [2]:
# Fixed locations
FSLBINDIR  = '/usr/local/fsl/bin'
FSLMATHS   = join(FSLBINDIR, 'fslmaths')
FSLSTATS   = join(FSLBINDIR, 'fslstats')
AROMADIR   = os.path.abspath('../icaaroma/data')

Original ICA-AROMA code. NB. Depends on working directory and hacks construction of path to fslinfo.

In [3]:
# commands module is deprecated since 2.6 and removed in 3.X so patch in a replacement
from sys import version_info
if version_info < (3, 0):
    from commands import getoutput
else:
    # legacy function resembling commands one
    from subprocess import getoutput

def feature_spatial_orig(fslDir, tempDir, aromaDir, melIC):
	""" This function extracts the spatial feature scores. For each IC it determines the fraction of the mixture modeled thresholded Z-maps respecitvely located within the CSF or at the brain edges, using predefined standardized masks.

	Parameters
	---------------------------------------------------------------------------------
	fslDir:		Full path of the bin-directory of FSL
	tempDir:	Full path of a directory where temporary files can be stored (called 'temp_IC.nii.gz')
	aromaDir:	Full path of the ICA-AROMA directory, containing the mask-files (mask_edge.nii.gz, mask_csf.nii.gz & mask_out.nii.gz) 
	melIC:		Full path of the nii.gz file containing mixture-modeled threholded (p>0.5) Z-maps, registered to the MNI152 2mm template
	
	Returns
	---------------------------------------------------------------------------------
	edgeFract:	Array of the edge fraction feature scores for the components of the melIC file
	csfFract:	Array of the CSF fraction feature scores for the components of the melIC file"""

	# Get the number of ICs
	numICs = int(getoutput('%sfslinfo %s | grep dim4 | head -n1 | awk \'{print $2}\'' % (fslDir, melIC) ))

	# Loop over ICs
	edgeFract=np.zeros(numICs)
	csfFract=np.zeros(numICs)
	for i in range(0,numICs):
		# Define temporary IC-file
		tempIC = os.path.join(tempDir,'temp_IC.nii.gz')

		# Extract IC from the merged melodic_IC_thr2MNI2mm file
		os.system(' '.join([os.path.join(fslDir,'fslroi'),
			melIC,
			tempIC,
			str(i),
			'1']))

		# Change to absolute Z-values
		os.system(' '.join([os.path.join(fslDir,'fslmaths'),
			tempIC,
			'-abs',
			tempIC]))
		
		# Get sum of Z-values within the total Z-map (calculate via the mean and number of non-zero voxels)
		totVox = int(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-V | awk \'{print $1}\''])))
		
		if not (totVox == 0):
			totMean = float(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-M'])))
		else:
			print('     - The spatial map of component ' + str(i+1) + ' is empty. Please check!')
			totMean = 0

		totSum = totMean * totVox
		
		# Get sum of Z-values of the voxels located within the CSF (calculate via the mean and number of non-zero voxels)
		csfVox = int(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-k mask_csf.nii.gz',
							'-V | awk \'{print $1}\''])))

		if not (csfVox == 0):
			csfMean = float(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-k mask_csf.nii.gz',
							'-M'])))
		else:
			csfMean = 0

		csfSum = csfMean * csfVox	

		# Get sum of Z-values of the voxels located within the Edge (calculate via the mean and number of non-zero voxels)
		edgeVox = int(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-k mask_edge.nii.gz',
							'-V | awk \'{print $1}\''])))
		if not (edgeVox == 0):
			edgeMean = float(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-k mask_edge.nii.gz',
							'-M'])))
		else:
			edgeMean = 0
		
		edgeSum = edgeMean * edgeVox

		# Get sum of Z-values of the voxels located outside the brain (calculate via the mean and number of non-zero voxels)
		outVox = int(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-k mask_out.nii.gz',
							'-V | awk \'{print $1}\''])))
		if not (outVox == 0):
			outMean = float(getoutput(' '.join([os.path.join(fslDir,'fslstats'),
							tempIC,
							'-k mask_out.nii.gz',
							'-M'])))
		else:
			outMean = 0
		
		outSum = outMean * outVox

		# Determine edge and CSF fraction
		if not (totSum == 0):
			edgeFract[i] = (outSum + edgeSum)/(totSum - csfSum)
			csfFract[i] = csfSum / totSum
		else:
			edgeFract[i]=0
			csfFract[i]=0

	# Remove the temporary IC-file
	os.remove(tempIC)

	# Return feature scores
	return edgeFract, csfFract

Wrap it up a bit to get it in a usable form.

In [4]:
import shutil
import contextlib
import os

@contextlib.contextmanager
def working_directory(path):
    """Temporarily change working directory."""
    prev_cwd = os.getcwd()
    os.chdir(path)
    try:
        yield
    finally:
        os.chdir(prev_cwd)
        
def feature_spatial_orig_wrapped(melodic_ic_file, aroma_dir=AROMADIR):
    aroma_dir = os.path.abspath(aroma_dir)
    melodic_ic_file = os.path.abspath(melodic_ic_file)
    tempdir = mkdtemp()
    with working_directory(aroma_dir):
        edge_fraction, csf_fraction = feature_spatial_orig(
            fslDir=FSLBINDIR + '/',
            tempDir=tempdir,
            aromaDir=aroma_dir,
            melIC=melodic_ic_file
        )

    shutil.rmtree(tempdir)
    return edge_fraction, csf_fraction

In [5]:
feat_orig = feature_spatial_orig_wrapped(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')

The first refactored version with separate calls to new function `zsum` and still using `fslstats`. 

In [6]:
def zsums_0(filename, mask=None):
    """Sum of Z-values within the total Z-map or within a subset defined by a mask.

    Calculated via the mean and number of non-zero voxels.

    Parameters
    ----------
    filename: str
        zmap nifti file
    mask: Optional(str)
        mask file

    Returns
    -------
    numpy array
        sums of pixels across the whole images or just within the mask
    """
    assert isfile(filename)

    _, tmpfile = mkstemp(prefix='zsums', suffix='.nii.gz')

    # Change to absolute Z-values
    call([FSLMATHS, filename, '-abs', tmpfile])

    preamble = [FSLSTATS, '-t', tmpfile]
    if mask is not None:
        preamble += ['-k', mask]
    counts_cmd = preamble + ['-V']
    means_cmd  = preamble + ['-M']

    p = Popen(counts_cmd, stdout=PIPE)
    counts = np.loadtxt(p.stdout)[:, 0]
    p = Popen(means_cmd, stdout=PIPE)
    means = np.loadtxt(p.stdout)

    os.unlink(tmpfile)
    return means * counts


def feature_spatial_0(melodic_ic_file, aroma_dir=None):
    """Spatial feature scores.

    For each IC determine the fraction of the mixture modelled thresholded Z-maps respectively located within
    the CSF or at the brain edges, using predefined standardized masks.

    Parameters
    ----------
    melodic_ic_file: str
        nii.gz file containing mixture-modelled thresholded (p>0.5) Z-maps, registered to the MNI152 2mm template
    aroma_dir:  str
        ICA-AROMA directory, containing the mask-files (mask_edge.nii.gz, mask_csf.nii.gz & mask_out.nii.gz)

    Returns
    -------
    tuple (rank 1 array like, rank 1 array like)
        Edge and CSF fraction feature scores for the components of the melodic_ics_file file
    """
    assert isfile(melodic_ic_file)
    if aroma_dir is None:
        aroma_dir = AROMADIR
    assert isdir(aroma_dir)

    edge_mask = join(aroma_dir, 'mask_edge.nii.gz')
    csf_mask  = join(aroma_dir, 'mask_csf.nii.gz')
    out_mask  = join(aroma_dir, 'mask_out.nii.gz')

    total_sum = zsums_0(melodic_ic_file)
    csf_sum = zsums_0(melodic_ic_file, mask=csf_mask)
    edge_sum = zsums_0(melodic_ic_file, mask=edge_mask)
    outside_sum = zsums_0(melodic_ic_file, mask=out_mask)

    edge_fraction = np.where(total_sum > csf_sum, (outside_sum + edge_sum) / (total_sum - csf_sum), 0)
    csf_fraction = np.where(total_sum > csf_sum, csf_sum / total_sum, 0)

    return edge_fraction, csf_fraction

In [7]:
feat_0 = feature_spatial_0(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')
assert np.allclose(feat_0[0], feat_orig[0])
assert np.allclose(feat_0[1], feat_orig[1])

Combine the calls to `zsum` in to one to avoid some `fslmaths`/`fslstats` calls.

In [8]:
def zsums_1(filename, masks=None):
    """Sum of Z-values within the total Z-map or within a subset defined by a mask.

    Calculated via the mean and number of non-zero voxels.

    Parameters
    ----------
    filename: str
        zmap nifti file
    mask: Optional(str)
        mask file

    Returns
    -------
    numpy array
        sums of pixels across the whole images or just within the mask
    """
    assert isfile(filename)
    if masks is None:
        masks = [None]

    _, tmpfile = mkstemp(prefix='zsums', suffix='.nii.gz')

    # Change to absolute Z-values
    call([FSLMATHS, filename, '-abs', tmpfile])

    zsums = []
    for mask in masks:
        preamble = [FSLSTATS, '-t', tmpfile]
        if mask is not None:
            preamble += ['-k', mask]
        counts_cmd = preamble + ['-V']
        means_cmd  = preamble + ['-M']

        p = Popen(counts_cmd, stdout=PIPE)
        counts = np.loadtxt(p.stdout)[:, 0]
        p = Popen(means_cmd, stdout=PIPE)
        means = np.loadtxt(p.stdout)
        zsums.append(means * counts)
    os.unlink(tmpfile)
    return tuple(zsums)


def feature_spatial_1(melodic_ic_file, aroma_dir=None):
    """Spatial feature scores.

    For each IC determine the fraction of the mixture modelled thresholded Z-maps respectively located within
    the CSF or at the brain edges, using predefined standardized masks.

    Parameters
    ----------
    melodic_ic_file: str
        nii.gz file containing mixture-modelled thresholded (p>0.5) Z-maps, registered to the MNI152 2mm template
    aroma_dir:  str
        ICA-AROMA directory, containing the mask-files (mask_edge.nii.gz, mask_csf.nii.gz & mask_out.nii.gz)

    Returns
    -------
    tuple (rank 1 array like, rank 1 array like)
        Edge and CSF fraction feature scores for the components of the melodic_ics_file file
    """
    assert isfile(melodic_ic_file)
    if aroma_dir is None:
        aroma_dir = AROMADIR
    assert isdir(aroma_dir)

    csf_mask  = join(aroma_dir, 'mask_csf.nii.gz')
    edge_mask = join(aroma_dir, 'mask_edge.nii.gz')
    out_mask  = join(aroma_dir, 'mask_out.nii.gz')

    total_sum, csf_sum, edge_sum, outside_sum = zsums_1(
        melodic_ic_file, masks=[None, csf_mask, edge_mask, out_mask]
    )

    edge_fraction = np.where(total_sum > csf_sum, (outside_sum + edge_sum) / (total_sum - csf_sum), 0)
    csf_fraction = np.where(total_sum > csf_sum, csf_sum / total_sum, 0)

    return edge_fraction, csf_fraction

In [9]:
feat_1 = feature_spatial_1(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')
assert np.allclose(feat_1[0], feat_0[0])
assert np.allclose(feat_1[1], feat_0[1])

Use nibabel to get files as numpy arrays and do operations in numpy.

In [10]:
import nibabel as nib

def zsums_2(filename, maskfiles=None):
    """Sum of Z-values within the total Z-map or within a subset defined by a mask.

    Calculated via the mean and number of non-zero voxels.

    Parameters
    ----------
    filename: str
        zmap nifti file
    mask: Optional(str)
        mask file

    Returns
    -------
    numpy array
        sums of pixels across the whole images or just within the mask
    """
    assert isfile(filename)
    if maskfiles is None:
        maskfiles = [None]

    img = np.abs(nib.load(filename).get_data())

    masks = [
        nib.load(fname).get_data().astype('bool')[..., None] if fname is not None else np.ones_like(img)
        for fname in maskfiles
    ]
    zsums = [(img * mask).sum(axis=(0, 1, 2)) for mask in masks]

    return tuple(zsums)


def feature_spatial_2(melodic_ic_file, aroma_dir=None):
    """Spatial feature scores.

    For each IC determine the fraction of the mixture modelled thresholded Z-maps respectively located within
    the CSF or at the brain edges, using predefined standardized masks.

    Parameters
    ----------
    melodic_ic_file: str
        nii.gz file containing mixture-modelled thresholded (p>0.5) Z-maps, registered to the MNI152 2mm template
    aroma_dir:  str
        ICA-AROMA directory, containing the mask-files (mask_edge.nii.gz, mask_csf.nii.gz & mask_out.nii.gz)

    Returns
    -------
    tuple (rank 1 array like, rank 1 array like)
        Edge and CSF fraction feature scores for the components of the melodic_ics_file file
    """
    assert isfile(melodic_ic_file)
    if aroma_dir is None:
        aroma_dir = AROMADIR
    assert isdir(aroma_dir)

    csf_mask  = join(aroma_dir, 'mask_csf.nii.gz')
    edge_mask = join(aroma_dir, 'mask_edge.nii.gz')
    out_mask  = join(aroma_dir, 'mask_out.nii.gz')

    total_sum, csf_sum, edge_sum, outside_sum = zsums_2(
        melodic_ic_file, maskfiles=[None, csf_mask, edge_mask, out_mask]
    )

    edge_fraction = np.where(total_sum > csf_sum, (outside_sum + edge_sum) / (total_sum - csf_sum), 0)
    csf_fraction = np.where(total_sum > csf_sum, csf_sum / total_sum, 0)

    return edge_fraction, csf_fraction

In [11]:
feat_2 = feature_spatial_2(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')
assert np.allclose(feat_2[0], feat_1[0])
assert np.allclose(feat_2[1], feat_1[1])

In [12]:
%timeit feature_spatial_orig_wrapped(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')

%timeit feature_spatial_0(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')

%timeit feature_spatial_1(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')

%timeit feature_spatial_2(melodic_ic_file='../test/refout/melodic_IC_thr_MNI2mm.nii.gz')

1 loop, best of 3: 2min 5s per loop
1 loop, best of 3: 1min 11s per loop
1 loop, best of 3: 42.6 s per loop
1 loop, best of 3: 3.08 s per loop


OK, we've quite a significant speed up here (about 50x), mainly by doing the masking etc in process with numpy instead of making external calls to FSL routines.